In [4]:
import os
import csv
import time
import socket
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import yt_dlp
from googletrans import Translator
from datetime import datetime
import signal
import sys
import pandas as pd

In [5]:

# 初始化翻译器
translator = Translator()

# 检查网络连接
def check_network():
    try:
        socket.create_connection(("8.8.8.8", 53), timeout=5)
        return True
    except OSError:
        return False

# 获取 YouTube 视频链接（支持单个视频和合集）
def get_playlist_videos(url):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'playlist_items': '1-1000',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=False)
        if 'entries' in result:
            return [(entry['url'], entry['title']) for entry in result['entries']]  # Return URL and title
        return [(url, None)]  # Single video case

# 自动获取 YouTube 的 cookies
def get_youtube_cookies(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    options = webdriver.EdgeOptions()
    options.add_argument('--headless')
    
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    driver.get("https://www.youtube.com")
    time.sleep(5)
    
    cookies = driver.get_cookies()
    driver.quit()
    
    cookies_file = os.path.join(output_dir, "cookies.txt")
    with open(cookies_file, "w") as f:
        for cookie in cookies:
            f.write(f"{cookie['name']}={cookie['value']}\n")
    return cookies_file

# 检查视频是否已存在（基于标题）
def video_exists(video_title, output_dir):
    sanitized_title = ''.join(c for c in video_title if c.isalnum() or c in ' -_')  # Sanitize for filename
    for file in os.listdir(output_dir):
        if sanitized_title in file:
            return True
    return False

# 下载视频并获取信息
def download_video_and_get_info(video_url, video_title, index, cookies_file, output_dir, max_retries=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    if not os.path.exists(cookies_file):
        print(f"未找到 {cookies_file}，尝试重新生成...")
        cookies_file = get_youtube_cookies(output_dir)
    
    # Use index and title for filename
    filename_template = os.path.join(output_dir, f"{index:03d}|%(title)s.%(ext)s")
    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'merge_output_format': 'mp4',
        'outtmpl': filename_template,
        'cookies': cookies_file,
        'quiet': True,
        'retries': 10,
        'fragment_retries': 10,
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
        'postprocessor_args': [
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-b:a', '192k',
        ],
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for attempt in range(max_retries):
            if not check_network():
                print(f"网络不可用，请检查连接。等待 10 秒后重试 (尝试 {attempt + 1}/{max_retries})...")
                time.sleep(10)
                continue
            
            try:
                print(f"开始下载视频: {video_url} (尝试 {attempt + 1}/{max_retries})")
                info = ydl.extract_info(video_url, download=False)
                
                # Use pre-fetched title if available, otherwise fetch from info
                original_title = video_title if video_title else info.get('title', 'N/A')
                if video_exists(original_title, output_dir):
                    print(f"视频 '{original_title}' 已存在，跳过下载")
                    return None
                
                ydl.download([video_url])
                
                chinese_title = translator.translate(original_title, dest='zh-cn').text
                language = info.get('language', 'N/A')
                resolution = info.get('resolution', 'N/A') or f"{info.get('height', 'N/A')}p"
                format_type = 'mp4'
                upload_date = info.get('upload_date', 'N/A')
                if upload_date != 'N/A':
                    upload_date = datetime.strptime(upload_date, '%Y%m%d').strftime('%Y-%m-%d')
                uploader = info.get('uploader', 'N/A')
                
                return {
                    '原标题': original_title,
                    '中文标题': chinese_title,
                    '语言': language,
                    '分辨率': resolution,
                    '格式': format_type,
                    '发布日期': upload_date,
                    '来源': uploader
                }
            except (yt_dlp.utils.ExtractorError, yt_dlp.utils.DownloadError) as e:
                print(f"下载 {video_url} 失败: {str(e)}")
                if attempt < max_retries - 1:
                    print(f"等待 5 秒后重试...")
                    time.sleep(5)
                else:
                    print(f"达到最大重试次数 ({max_retries})，跳过视频 {video_url}")
                    return None

# 追加写入 CSV 文件并打印内容
def append_to_csv(video_info, csv_output):
    output_dir = os.path.dirname(csv_output)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    file_exists = os.path.exists(csv_output)
    
    with open(csv_output, 'a', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['原标题', '中文标题', '语言', '分辨率', '格式', '发布日期', '来源']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()
        
        writer.writerow(video_info)
    
    print(f"\n更新 CSV 文件: {csv_output}")
    df = pd.read_csv(csv_output, encoding='utf-8-sig')
    print(df.to_string(index=False))
    print("\n")

# 处理中断信号
def signal_handler(sig, frame, csv_output):
    print("\n收到中断信号（Ctrl+C），程序退出...")
    sys.exit(0)

# 主函数
def main(playlist_url, video_output_dir="D:/Videos", csv_output="D:/Data/video_info.csv"):
    if not check_network():
        print("网络不可用，请检查你的互联网连接后重试。")
        return
    
    cookies_file = get_youtube_cookies(video_output_dir)
    video_urls = get_playlist_videos(playlist_url)
    video_info_list = []
    
    signal.signal(signal.SIGINT, lambda sig, frame: signal_handler(sig, frame, csv_output))
    
    for index, (url, title) in enumerate(video_urls, start=1):  # Start index at 1
        print(f"处理视频 {index}: {url}")
        video_info = download_video_and_get_info(url, title, index, cookies_file, video_output_dir)
        if video_info:
            video_info_list.append(video_info)
            append_to_csv(video_info, csv_output)



In [6]:

if __name__ == "__main__":
    playlist_url = "https://www.youtube.com/playlist?list=PL3HhMTCWB-e8dxlpnMDQMFEjX1TRDMsnF"
    video_output_dir = "D:/Videos"
    csv_output_path = "D:/Data/video_info.csv"
    main(playlist_url, video_output_dir=video_output_dir, csv_output=csv_output_path)

处理视频 1: https://www.youtube.com/watch?v=DVPxMeQcntw
开始下载视频: https://www.youtube.com/watch?v=DVPxMeQcntw (尝试 1/3)
                                                            
更新 CSV 文件: D:/Data/video_info.csv
                                                                                              原标题                              中文标题 语言       分辨率  格式       发布日期         来源
Israel Hezbollah War LIVE | UN Security Council Meets In Emergency Session Over Lebanon Explosion 以色列真主党战争现场直播|联合国安理会在黎巴嫩爆炸的紧急会议上开会 en 1920x1080 mp4 2024-09-21 CNN-News18


处理视频 2: https://www.youtube.com/watch?v=S2S29QWK4vk
开始下载视频: https://www.youtube.com/watch?v=S2S29QWK4vk (尝试 1/3)
                                                            
更新 CSV 文件: D:/Data/video_info.csv
                                                                                                原标题                                     中文标题 语言       分辨率  格式       发布日期         来源
  Israel Hezbollah War LIVE | UN Security Council Meets In E

ERROR: [youtube] RhzcgL9cEjo: Requested format is not available. Use --list-formats for a list of available formats


下载 https://www.youtube.com/watch?v=RhzcgL9cEjo 失败: ERROR: [youtube] RhzcgL9cEjo: Requested format is not available. Use --list-formats for a list of available formats
等待 5 秒后重试...
开始下载视频: https://www.youtube.com/watch?v=RhzcgL9cEjo (尝试 2/3)


ERROR: [youtube] RhzcgL9cEjo: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


下载 https://www.youtube.com/watch?v=RhzcgL9cEjo 失败: ERROR: [youtube] RhzcgL9cEjo: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
等待 5 秒后重试...
开始下载视频: https://www.youtube.com/watch?v=RhzcgL9cEjo (尝试 3/3)


...

ERROR: [youtube] E0yn0HT0Yi0: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


下载 https://www.youtube.com/watch?v=E0yn0HT0Yi0 失败: ERROR: [youtube] E0yn0HT0Yi0: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
达到最大重试次数 (3)，跳过视频 https://www.youtube.com/watch?v=E0yn0HT0Yi0
处理视频 26: https://www.youtube.com/watch?v=BrdmgvTu2YM
开始下载视频: https://www.youtube.com/watch?v=BrdmgvTu2YM (尝试 1/3)


ERROR: [youtube] BrdmgvTu2YM: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


下载 https://www.youtube.com/watch?v=BrdmgvTu2YM 失败: ERROR: [youtube] BrdmgvTu2YM: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
等待 5 秒后重试...
开始下载视频: https://www.youtube.com/watch?v=BrdmgvTu2YM (尝试 2/3)


ReadTimeout: timed out